In [15]:
import cv2 as cv
import numpy as np
import mysql.connector
import datetime
import dlib
import face_recognition as fr
import os


def compare(face_list, face_encoding, nof):
    #Face_list is a list of unique-face encodings
    for i in face_list:
        #Compare the current face(face_encoding) with every encoding in face_list
        #Comparison_list is a list of true false values. True if face already exists in face_list
        comparison_list = fr.compare_faces([i], face_encoding, tolerance = 0.5)
       
        for element in comparison_list:
           #If a matching face is found, return true
            if element == True:
                return True
           
    return False


#Initialise Camera
vid = cv.VideoCapture(0)
dt=datetime.date.today()

# connecting to the db
mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="cisco_thingqbator"
    )
db_cursor=mydb.cursor()

#Create a cascade classifier object
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
faces=[]

#No. of frames
nof = 0

#List to store each face, to be done later
face_list=[]

folder = "images/"
total_count = db_cursor.execute("SELECT Count FROM primary_daywise WHERE Date=(%s)", (dt))
mydb.commit()
if total_count is not None:
        count=total_count
else:
    count=0


while True:
    #checks if the day hasn't changed or if the month has changed
    now=datetime.date.today()
    if dt.day+1==now.day or dt.day>now.day:
        
        count += len(face_list)
        if total_count is None:
            db_cursor.execute("INSERT INTO primary_daywise(Date, Count) VALUES(%s,%s)",(now, count))
            mydb.commit()
            print(db_cursor.rowcount, "Record Inserted")
            #Empty face list on a new day
            face_list=[]
            dt = now
        elif total_count is not None:
            db_cursor.execute("UPDATE primary_daywise SET Count = (%s) WHERE Date=(%d)", (count, now))
            mydb.commit()
            print("Record Updated")
    
       

    #check returns bool, frame returns the image as numpy array
    check, frame = vid.read()
    if not check:
        continue
 
   
   
    #Convert image to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
   
    #Detect multiple faces. Returns a list of faces in the current frame.
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)
   
    #If frame has no face detected, continue loop.
    if len(faces)==0:
        continue
   
    #Increment number of frames
    nof += 1
       
    #Draw a rectangle around each face
    for top, left, bottom, right in faces:
        gray = cv.rectangle(gray, (top, left), (top+bottom, left+right), (0,255,0), 1)
       
   
    #Process only if faces is not empty
    if len(faces)!=0:
        for face in faces[:1]:
            x, y, w, h = face
           
            # Get the face ROI(region of interest)
            offset = 7
            face_section = gray[y-offset:y+h+offset, x-offset:x+w+offset]
            face_section = cv.resize(face_section, (100, 100))
           
            #Write current face to local folder as jpg image
            cv.imwrite(os.path.join(folder,"frame{:d}.jpg".format(nof)), face_section)
           
            #Load the face as jpg image
            image = fr.load_image_file( "images/frame{:d}.jpg".format(nof))
           
        
            #Convert image into encoding
            if len(fr.face_encodings(image))!=0:
                face_encoding = fr.face_encodings(image)[0]
               
                #If face_list is empty or current face is unique, append its encoding
                if len(face_list)==0 or compare(face_list, face_encoding, nof) == False:
                    face_list.append(face_encoding)
                    print("appended")
                     
               
               
    #Show the faces with rectangle drawn on them
    cv.imshow("Face", face_section)
   
    #Wait for 1ms per frame. Quit if 'q' pressed.
    key = cv.waitKey(1)
    if key == ord('q'):
        break
   
   
#Print no. of frames
print("No. of frames:", nof)

#Print no. of unique faces
print("No. of faces:", len(face_list))

#putting no. of unique faces in count
count += len(face_list)

today=datetime.date.today()
#sql_query = "INSERT INTO primary_daywise VALUES(dt, count)"
if total_count is None:
    db_cursor.execute("INSERT INTO primary_daywise(Date, Count) VALUES(%s,%s)",(today, count))
    mydb.commit()
    print(db_cursor.rowcount, "Record Inserted")
elif total_count is not None:
    db_cursor.execute("UPDATE primary_daywise SET Count = (%s) WHERE Date=(%s)", (count, today))
    mydb.commit()
    print("Record Updated")
    
# db_cursor.execute("INSERT INTO primary_daywise(Date, Count) VALUES(%s,%s)",(today, count))
# mydb.commit()
# print(db_cursor.rowcount, "Record is Inserted")
vid.release()
cv.destroyAllWindows()

ValueError: Could not process parameters

import mysql.connector
create table TESTBLOB(framenum int not null, feature blob not null);